In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [2]:
def setup_download_from_s3( rel_fp, recursive=True ):
    s3_fp = 's3://mousebrainatlas-data/'+rel_fp
    local_fp = os.environ['ROOT_DIR']+rel_fp

    if os.path.exists(local_fp):
        print('ALREADY DOWNLOADED FILE')
        return 
    
    if recursive:
        ! aws s3 cp --recursive $s3_fp $local_fp
    else:
        ! aws s3 cp $s3_fp $local_fp

In [3]:
setup_download_from_s3('CSHL_new_regions/MD589/')

ALREADY DOWNLOADED FILE


In [2]:
#credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
credFiles= '/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


Connecting kui@ucsd-demo-db.datajoint.io:3306


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [3]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [4]:
stack = 'MD594'
#yaml_file = 'shape_params-aws.yaml'
yaml_file = 'shape_params.yaml'
patch_file = 'CSHL_patches_200'
scripts_dir = os.environ['REPO_DIR']

In [5]:
Structure4.drop()

`kui_diffusionmap`.`structure4` (27 tuples)
Proceed? [yes, No]: No


In [5]:
@schema
class Structure6(dj.Manual):
    definition = """
    id  : int   # sequence
    --------
    structure       : char(10)    # name of mouse brain's structure
    positive_s3_fp  : varchar(200)  # (str) file path
    negative_s3_fp  : varchar(200)  # (str) file path
    """

In [6]:
paired_structures = ['5N', '6N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', \
                     'SNC', 'SNR', '3N', '4N', 'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC'] #, '7N'
all_structures = paired_structures + singular_structures
s3_fp = patch_file+'_features/'+stack+'/' #s3://mousebrainatlas-data/
print(s3_fp)

CSHL_patches_200_features/MD594/


In [7]:
n=0
for struc in all_structures:
    print("\nAdding "+struc+' to the database')
    Structure6.insert1(dict(id=n,
                           structure=struc,
                           positive_s3_fp=s3_fp+struc+'/'+stack+'_'+struc+'_positive.pkl',
                           negative_s3_fp=s3_fp+struc+'/'+stack+'_'+struc+'_negative.pkl',)
                      ,skip_duplicates=False)
    n+=1


Adding 5N to the database

Adding 6N to the database

Adding 7n to the database

Adding Amb to the database

Adding LC to the database

Adding LRt to the database

Adding Pn to the database

Adding Tz to the database

Adding VLL to the database

Adding RMC to the database

Adding SNC to the database

Adding SNR to the database

Adding 3N to the database

Adding 4N to the database

Adding Sp5I to the database

Adding Sp5O to the database

Adding Sp5C to the database

Adding PBG to the database

Adding 10N to the database

Adding VCA to the database

Adding VCP to the database

Adding DC to the database

Adding AP to the database

Adding 12N to the database

Adding RtTg to the database

Adding SC to the database

Adding IC to the database


In [8]:
structureTable = Structure6.fetch(as_dict=True)
strucDF = pd.DataFrame(structureTable)
strucDF

,id,structure,positive_s3_fp,negative_s3_fp
0,0,5N,CSHL_patches_200_features/MD594/5N/MD594_5N_po...,CSHL_patches_200_features/MD594/5N/MD594_5N_ne...
1,1,6N,CSHL_patches_200_features/MD594/6N/MD594_6N_po...,CSHL_patches_200_features/MD594/6N/MD594_6N_ne...
2,2,7n,CSHL_patches_200_features/MD594/7n/MD594_7n_po...,CSHL_patches_200_features/MD594/7n/MD594_7n_ne...
3,3,Amb,CSHL_patches_200_features/MD594/Amb/MD594_Amb_...,CSHL_patches_200_features/MD594/Amb/MD594_Amb_...
4,4,LC,CSHL_patches_200_features/MD594/LC/MD594_LC_po...,CSHL_patches_200_features/MD594/LC/MD594_LC_ne...
5,5,LRt,CSHL_patches_200_features/MD594/LRt/MD594_LRt_...,CSHL_patches_200_features/MD594/LRt/MD594_LRt_...
6,6,Pn,CSHL_patches_200_features/MD594/Pn/MD594_Pn_po...,CSHL_patches_200_features/MD594/Pn/MD594_Pn_ne...
7,7,Tz,CSHL_patches_200_features/MD594/Tz/MD594_Tz_po...,CSHL_patches_200_features/MD594/Tz/MD594_Tz_ne...
8,8,VLL,CSHL_patches_200_features/MD594/VLL/MD594_VLL_...,CSHL_patches_200_features/MD594/VLL/MD594_VLL_...
9,9,RMC,CSHL_patches_200_features/MD594/RMC/MD594_RMC_...,CSHL_patches_200_features/MD594/RMC/MD594_RMC_...


In [14]:
Process5.drop()

`kui_diffusionmap`.`__process5` (0 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [9]:
@schema
class Process6(dj.Computed):
    definition="""
    -> Structure6
    -----
    size_positive_properties : int   #size of file
    size_negative_properties : int   #size of file
    """

    bucket = "mousebrainatlas-data"
    client = get_s3_client(credFiles)
    def make(self, key):
        struc = (Structure6 & key).fetch1('structure')
        print('populating for ', struc, end='\n')
        for state in ['positive', 'negative']:
            item_name = state+'_s3_fp'
            key_item = 'size_'+state+'_properties'
            s3_fp = (Structure6 & key).fetch1(item_name)
            try:
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size/1000)
            except:
                ! python3 $scripts_dir/Cell_generator.py $stack $struc $state $yaml_file $patch_file
                upload_fp = 's3://'+self.bucket+'/'+s3_fp
                pkl_out_file = os.environ['ROOT_DIR']+s3_fp
                ! aws s3 cp $pkl_out_file $upload_fp
                print(pkl_out_file,upload_fp)
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size / 1000)
        try:
            self.insert1(key)
        except:
            print('could not insert key=', key)


./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


In [10]:
Process6.populate()

populating for  5N
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2019-06-27 21:47:46,574	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-27_21-47-46_98634/logs.
2019-06-27 21:47:46,680	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:26560 to respond...
2019-06-27 21:47:46,808	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:16410 to respond...
2019-06-27 21:47:46,810	INFO services.py:804 -- Starting Redis shard with 1.72 GB max memory.
2019-06-27 21:47:46,820	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-27_21-47-46_98634/logs.
2019-06-27 21:47:46,821	INFO services.py:1427 -- Starting the Plasma object store with 2.58 GB memory using /tmp.
/U

/Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200_features/MD594/5N/MD594_5N_negative.pkl s3://mousebrainatlas-data/CSHL_patches_200_features/MD594/5N/MD594_5N_negative.pkl
populating for  6N
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2019-06-27 21:56:39,971	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-27_21-56-39_98709/logs.
2019-06-27 21:56:40,078	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:46508 to respond...
2019-06-27 21:56:40,204	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:20938 to respond...
2019-06-27 21:56:40,208	INFO services.py:804 -- Starting Redis shard with 1.72 GB max memory.
2019-06-27 21:56:40,219	INFO node.py:483 -- Process STDOUT and STDERR i

(pid=98793) 7n_positive 231 230 / 1000
(pid=98793) 7n_positive 241 240 / 1000
(pid=98793) 7n_positive 251 250 / 1000
(pid=98793) image /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200/MD594/7n/153_3.tif std= 9.614758073842161 too blank
(pid=98793) 7n_positive 261 260 / 1000
(pid=98793) image /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200/MD594/7n/152_4.tif std= 9.287022556671323 too blank
(pid=98793) 7n_positive 271 270 / 1000
(pid=98793) 7n_positive 281 280 / 1000
(pid=98793) 7n_positive 291 290 / 1000
(pid=98793) 7n_positive 301 300 / 1000
(pid=98793) 7n_positive 311 310 / 1000
(pid=98793) 7n_positive 321 320 / 1000
(pid=98793) 7n_positive 331 330 / 1000
(pid=98793) 7n_positive 341 340 / 1000
(pid=98793) 7n_positive 351 350 / 1000
(pid=98793) 7n_positive 361 360 / 1000
(pid=98793) 7n_positive 371 370 / 1000
(pid=98793) 7n_positive 381 380 / 1000
(pid=98793) 7n_positive 391 390 / 1000
(pid=98793) 7n_positive 401 400 / 1000
(pid=98793) 7n_positive 411 41

(pid=98831) 7n_negative 291 290 / 1000
(pid=98831) 7n_negative 301 300 / 1000
(pid=98831) 7n_negative 311 310 / 1000
(pid=98831) 7n_negative 321 320 / 1000
(pid=98831) 7n_negative 331 330 / 1000
(pid=98831) 7n_negative 341 340 / 1000
(pid=98831) 7n_negative 351 350 / 1000
(pid=98831) image /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200/MD594/7n_surround_500um_noclass/150_7.tif std= 8.296205857353655 too blank
(pid=98831) 7n_negative 361 360 / 1000
(pid=98831) 7n_negative 371 370 / 1000
(pid=98831) 7n_negative 381 380 / 1000
(pid=98831) 7n_negative 391 390 / 1000
(pid=98831) 7n_negative 401 400 / 1000
(pid=98831) 7n_negative 411 410 / 1000
(pid=98831) 7n_negative 421 420 / 1000
(pid=98831) 7n_negative 431 430 / 1000
(pid=98831) 7n_negative 441 440 / 1000
(pid=98831) 7n_negative 451 450 / 1000
(pid=98831) 7n_negative 461 460 / 1000
(pid=98831) 7n_negative 471 470 / 1000
(pid=98831) 7n_negative 481 480 / 1000
(pid=98831) 7n_negative 491 490 / 1000
(pid=98831) 7n_negative 

(pid=98907) Amb_negative 1 0 / 120
(pid=98907) Amb_negative 11 10 / 120
(pid=98907) Amb_negative 21 20 / 120
(pid=98907) Amb_negative 31 30 / 120
(pid=98907) Amb_negative 41 40 / 120
(pid=98907) Amb_negative 51 50 / 120
(pid=98907) image /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200/MD594/Amb_surround_500um_noclass/168_9.tif std= 7.505112635314286 too blank
(pid=98907) Amb_negative 61 60 / 120
(pid=98907) Amb_negative 71 70 / 120
(pid=98907) Amb_negative 81 80 / 120
(pid=98907) Amb_negative 91 90 / 120
(pid=98907) image /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200/MD594/Amb_surround_500um_noclass/170_4.tif std= 7.726096649527833 too blank
(pid=98907) Amb_negative 101 100 / 120
(pid=98907) Amb_negative 111 110 / 120
(pid=98907) Amb_negative 120
(pid=98907) Amb_negative finished in  63.3 seconds
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200_features/MD594/Amb/MD594_Amb_negative.pkl to s3://mousebrainatlas-data/CSHL_patches_200_f

...

(pid=979) RtTg_positive 41 40 / 799
(pid=979) RtTg_positive 51 50 / 799
(pid=979) RtTg_positive 61 60 / 799
(pid=979) RtTg_positive 71 70 / 799
(pid=979) RtTg_positive 81 80 / 799
(pid=979) RtTg_positive 91 90 / 799
(pid=979) RtTg_positive 101 100 / 799
(pid=979) RtTg_positive 111 110 / 799
(pid=979) RtTg_positive 121 120 / 799
(pid=979) RtTg_positive 131 130 / 799
(pid=979) RtTg_positive 141 140 / 799
(pid=979) RtTg_positive 151 150 / 799
(pid=979) RtTg_positive 161 160 / 799
(pid=979) RtTg_positive 171 170 / 799
(pid=979) RtTg_positive 181 180 / 799
(pid=979) RtTg_positive 191 190 / 799
(pid=979) RtTg_positive 201 200 / 799
(pid=979) RtTg_positive 211 210 / 799
(pid=979) RtTg_positive 221 220 / 799
(pid=979) RtTg_positive 231 230 / 799
(pid=979) RtTg_positive 241 240 / 799
(pid=979) RtTg_positive 251 250 / 799
(pid=979) RtTg_positive 261 260 / 799
(pid=979) RtTg_positive 271 270 / 799
(pid=979) RtTg_positive 281 280 / 799
(pid=979) RtTg_positive 291 290 / 799
(pid=979) RtTg_positive 

(pid=1020) RtTg_negative 391 390 / 800
(pid=1020) RtTg_negative 401 400 / 800
(pid=1020) RtTg_negative 411 410 / 800
(pid=1020) RtTg_negative 421 420 / 800
(pid=1020) RtTg_negative 431 430 / 800
(pid=1020) RtTg_negative 441 440 / 800
(pid=1020) RtTg_negative 451 450 / 800
(pid=1020) RtTg_negative 461 460 / 800
(pid=1020) RtTg_negative 471 470 / 800
(pid=1020) RtTg_negative 481 480 / 800
(pid=1020) RtTg_negative 491 490 / 800
(pid=1020) RtTg_negative 501 500 / 800
(pid=1020) RtTg_negative 511 510 / 800
(pid=1020) RtTg_negative 521 520 / 800
(pid=1020) RtTg_negative 531 530 / 800
(pid=1020) RtTg_negative 541 540 / 800
(pid=1020) RtTg_negative 551 550 / 800
(pid=1020) RtTg_negative 561 560 / 800
(pid=1020) RtTg_negative 571 570 / 800
(pid=1020) RtTg_negative 581 580 / 800
(pid=1020) RtTg_negative 591 590 / 800
(pid=1020) RtTg_negative 601 600 / 800
(pid=1020) RtTg_negative 611 610 / 800
(pid=1020) RtTg_negative 621 620 / 800
(pid=1020) RtTg_negative 631 630 / 800
(pid=1020) RtTg_negative 

(pid=1066) SC_positive 801 800 / 1000
(pid=1066) SC_positive 811 810 / 1000
(pid=1066) SC_positive 821 820 / 1000
(pid=1066) SC_positive 831 830 / 1000
(pid=1066) SC_positive 841 840 / 1000
(pid=1066) SC_positive 851 850 / 1000
(pid=1066) SC_positive 861 860 / 1000
(pid=1066) SC_positive 871 870 / 1000
(pid=1066) SC_positive 881 880 / 1000
(pid=1066) SC_positive 891 890 / 1000
(pid=1066) SC_positive 901 900 / 1000
(pid=1066) SC_positive 911 910 / 1000
(pid=1066) SC_positive 921 920 / 1000
(pid=1066) SC_positive 931 930 / 1000
(pid=1066) SC_positive 941 940 / 1000
(pid=1066) SC_positive 951 950 / 1000
(pid=1066) SC_positive 961 960 / 1000
(pid=1066) SC_positive 971 970 / 1000
(pid=1066) SC_positive 981 980 / 1000
(pid=1066) SC_positive 991 990 / 1000
(pid=1066) SC_positive 1000
(pid=1066) SC_positive finished in 615.5 seconds
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200_features/MD594/SC/MD594_SC_positive.pkl to s3://mousebrainatlas-data/CSHL_patches_200_features

/Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200_features/MD594/SC/MD594_SC_negative.pkl s3://mousebrainatlas-data/CSHL_patches_200_features/MD594/SC/MD594_SC_negative.pkl
populating for  IC
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2019-06-28 02:16:12,411	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-28_02-16-12_1138/logs.
2019-06-28 02:16:12,521	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:53745 to respond...
2019-06-28 02:16:12,647	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:24406 to respond...
2019-06-28 02:16:12,650	INFO services.py:804 -- Starting Redis shard with 1.72 GB max memory.
2019-06-28 02:16:12,662	INFO node.py:483 -- Process STDOUT and STDERR is

(pid=1191) IC_negative 1 0 / 1000
(pid=1191) IC_negative 11 10 / 1000
(pid=1191) IC_negative 21 20 / 1000
(pid=1191) IC_negative 31 30 / 1000
(pid=1191) IC_negative 41 40 / 1000
(pid=1191) IC_negative 51 50 / 1000
(pid=1191) IC_negative 61 60 / 1000
(pid=1191) IC_negative 71 70 / 1000
(pid=1191) IC_negative 81 80 / 1000
(pid=1191) IC_negative 91 90 / 1000
(pid=1191) IC_negative 101 100 / 1000
(pid=1191) IC_negative 111 110 / 1000
(pid=1191) IC_negative 121 120 / 1000
(pid=1191) IC_negative 131 130 / 1000
(pid=1191) IC_negative 141 140 / 1000
(pid=1191) IC_negative 151 150 / 1000
(pid=1191) IC_negative 161 160 / 1000
(pid=1191) IC_negative 171 170 / 1000
(pid=1191) IC_negative 181 180 / 1000
(pid=1191) IC_negative 191 190 / 1000
(pid=1191) IC_negative 201 200 / 1000
(pid=1191) IC_negative 211 210 / 1000
(pid=1191) IC_negative 221 220 / 1000
(pid=1191) IC_negative 231 230 / 1000
(pid=1191) image /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_200/MD594/IC_surround_500um_nocla